In [95]:
merge_file = r'\\new-monster\T\freely_moving_ephys\ephys_recordings\101520\G6H28P6LT\fm1_hf1_ephys\101520_fm1_hf1_merge.mat'
config = {'ephys_sample_rate':  30000}

In [75]:
# package imports
import pandas as pd
import numpy as np
import xarray as xr
import os

# module imports
#from util.read_data import open_time
from scipy.io import loadmat

merge_info = loadmat(merge_file)
fileList = merge_info['fileList']
pathList = merge_info['pathList']
nSamps = merge_info['nSamps']

#load phy output data
phy_path = os.path.split(merge_file)
allSpikeT = np.load(os.path.join(phy_path[0],'spike_times.npy'))
clust = np.load(os.path.join(phy_path[0],'spike_clusters.npy'))
templates = np.load(os.path.join(phy_path[0],'templates.npy'))

# ephys_data_master holds information that is same for all recordings (i.e. cluster information + waveform)
ephys_data_master = pd.read_csv(os.path.join(phy_path[0],'cluster_info.tsv'),sep = '\t',index_col=0)

# insert waveforms
ephys_data_master['waveform'] = np.nan
ephys_data_master['waveform'] = ephys_data_master['waveform'].astype(object) # does this need to be an object? spikeT is an object because they are all diff length, but I don't think these need to be -cmn
for i, ind in enumerate(ephys_data_master.index):
    ephys_data_master.at[ind,'waveform'] = templates[ind,21:,ephys_data_master.at[ind,'ch']]

# create boundaries between recordings (in terms of timesamples)
boundaries = np.concatenate((np.array([0]),np.cumsum(nSamps)))

In [77]:
ephys_data_master

,Amplitude,ContamPct,KSLabel,amp,ch,depth,fr,group,n_spikes,sh,waveform
id,,,,,,,,,,,
0,392.0,19.8,mua,27.890654,0,1.0,6.634466,good,12812,0,"[0.003334756, 0.0074715107, 0.0053538, 0.00440..."
1,559.6,780.0,mua,25.195724,0,1.0,0.035213,noise,68,0,"[-0.009456127, -0.011458151, -0.01270871, -0.0..."
2,1338.4,311.9,mua,23.496183,0,1.0,0.012946,noise,25,0,"[-0.004573326, -0.007256208, -0.0097557865, -0..."
3,288.3,152.2,mua,23.235289,1,2.0,2.211144,noise,4270,0,"[0.012037983, -0.008592387, -0.0037987358, -0...."
4,237.8,157.2,mua,17.656380,9,10.0,0.924330,noise,1785,0,"[-0.017415067, -0.017093726, -0.024447942, -0...."
5,539.1,334.7,mua,23.930361,1,2.0,0.063693,noise,123,0,"[0.0034972217, -0.01611248, -0.014749196, -0.0..."
6,397.2,8.8,good,25.888235,2,3.0,3.971773,good,7670,0,"[0.0074969623, 0.0077951048, 0.008289589, 0.00..."
7,380.0,38.4,mua,28.836113,2,3.0,1.649296,noise,3185,0,"[0.016070712, 0.0009992011, 0.0051890896, 0.00..."
8,437.3,0.0,mua,30.644802,2,3.0,0.167260,noise,323,0,"[-0.0015187341, -0.00641259, -0.009183524, -0...."


In [123]:
# loop over each recording and create/save ephys_data for each one
for s in range(len(nSamps)+1):

    #select spikes in this timerange
    use = (allSpikeT >= boundaries[s]) & (allSpikeT<boundaries[s+1] )
    theseSpikes = allSpikeT[use]
    theseClust = clust[use[:,0]]

    #place spikes into ephys data structure
    ephys_data = ephys_data_master.copy()
    ephys_data['spikeT'] = np.NaN
    ephys_data['spikeT'] = ephys_data['spikeT'].astype(object)
    for c in np.unique(clust):
        ephys_data.at[c,'spikeT'] =theseSpikes[theseClust==c].flatten()/config['ephys_sample_rate'] 
    
    # get timestamp from csv for this recording
    fname = fileList[0,s][0].copy()
    fname = fname[0:-4] + '_BonsaiTS.csv'
    ephys_time_path = os.path.join(pathList[0,s][0],fname)
   # ephys_data['t0'] = open_time(ephys_time_path)[0]
    
    #write ephys data into json file
    fname = fileList[0,s][0].copy()
    fname = fname[0:-10] + '_ephys_merge.json'
    ephys_json_path = os.path.join(pathList[0,s][0],fname)
    ephys_data.to_json(ephys_json_path)

In [122]:
ephys_time_path

'\\\\new-monster\\t\\freely_moving_ephys\\ephys_recordings\\101520\\G6H28P6LT\\hf1_wn\\101520_G6H28P6LT_hf1_wn_Ephys_BonsaiTS.csv'